вот [эта статейка](https://blog.goodaudience.com/on-using-hyperopt-advanced-machine-learning-a2dde2ccece7) помогла  
Точно именно через `MongoTrials`, а не через `SparkTrials`?
Я не смог придумать ничего лучше, чем внутри каждого `objective` загружать нужную часть датасета и на ней оценивать.  
С одной стороны это вроде как масшатбируемо (докинул данные на новую машину и считаешь), однако точно ли это требовалось?

### Data prep part

In [1]:
import os
import sys

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.task.cpus", "8") \
    .config("spark.executor.cores", "8") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

COMMON_PATH = '/workspace/common'
sys.path.append(os.path.join(COMMON_PATH, 'utils'))
from processing import split_by_col

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [4]:
df = df.sample(False, 0.01)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [8]:
train_df.toPandas().to_csv(TRAIN_PATH + "_part_train.csv", index=False)
val_df.toPandas().to_csv(TRAIN_PATH + "_part_valid.csv", index=False)

In [9]:
sc.stop()

In [10]:
spark.stop()

### Hyperopt part

In [11]:
from hyperopt.mongoexp import MongoTrials
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
from objective import objective_xgb

In [14]:
! cp -av objective.py /usr/bin/

`objective.py' -> `/usr/bin/objective.py'


In [15]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 8,
    'silent': 0,
    'nworkers': 1
}

In [16]:
baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

In [17]:
trials = MongoTrials('mongo://localhost:27017/xgb_mongo/jobs', exp_key='exp1')

In [18]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


best = fmin(fn=objective_xgb,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

no last_id found, re-trying


 70%|███████   | 14/20 [03:12<01:22, 13.73s/trial, best loss: 0.5127516646730498]
CPU times: user 822 ms, sys: 68.9 ms, total: 891 ms
Wall time: 3min 36s


In [21]:
best = fmin(fn=objective_xgb,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

over-writing old domain trials attachment


100%|██████████| 20/20 [00:01<00:00, 19.89trial/s, best loss: 0.5127516646730498]


In [23]:
%%time
best = fmin(fn=objective_xgb,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

over-writing old domain trials attachment


100%|██████████| 20/20 [00:01<00:00, 19.88trial/s, best loss: 0.5127516646730498]
CPU times: user 40.8 ms, sys: 294 µs, total: 41.1 ms
Wall time: 1.05 s


Сохраняет историю! (не знаю почему не с первого раза досчитал до конца)